## **TextCNN**

In [26]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
from textblob import TextBlob
from wordcloud import WordCloud
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

In [27]:
meta_data = pd.read_csv("/kaggle/input/amazon32/amazon_metadata_health_personal_care.csv")
reviews = pd.read_csv("/kaggle/input/amazon32/amazon_reviews_health_personal_care.csv")

# Display first few rows
print("Metadata Dataset:")
display(meta_data.head())

print("\nReviews Dataset:")
display(reviews.head())


Metadata Dataset:


/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,main_category,title,average_rating,rating_number,price,store,parent_asin,date_first_available
0,Health & Personal Care,Silicone Bath Body Brush Exfoliator Shower Bac...,3.9,7,NaN,Rzoeox,B07V346GZH,"January 10, 2021"
1,Health & Personal Care,"iPhone 7 Plus 8 Plus Screen Protector, ZHXIN T...",3.8,2,NaN,ZHXIN,B075W927RH,"September 24, 2017"
2,Health & Personal Care,Zig Zag Rolling Machine 70mm Size With FREE BO...,3.9,7,NaN,NaN,B01FB26VKY,"May 27, 2016"
3,Health & Personal Care,Sting-Kill Disposable Wipes 8 Each ( Pack of 5),4.1,6,21.37,Sting-kill,B01IAI29RU,"July 11, 2016"
4,Health & Personal Care,Heated Eyelash Curler Mini Portable Electric E...,3.3,8,NaN,BiBOSS,B08CMN38RC,"July 24, 2020"



Reviews Dataset:


,rating,title,text,helpful_vote,parent_asin,verified_purchase,review_date
0,4.0,12 mg is 12 on the periodic table people! Mg f...,This review is more to clarify someone else’s ...,3,B07TDSJZMR,True,2020-02-06
1,5.0,Save the lanet using less plastic.,Love these easy multitasking bleach tablets. B...,3,B08637FWWF,True,2020-11-02
2,5.0,Fantastic,I have been suffering a couple months with hee...,0,B07KJVGNN5,True,2019-07-24
3,4.0,It holds the water and makes bubbles. That's ...,"It's cheap and it does what I wanted. The ""ma...",7,B092RP73CX,True,2022-09-04
4,1.0,Not for me,Didn't do a thing for me. Not saying they don'...,0,B08KYJLF5T,True,2022-01-20


In [28]:
# Convert review_date with error handling
reviews["review_date"] = pd.to_datetime(reviews["review_date"], errors='coerce', infer_datetime_format=True)

# Convert date_first_available with error handling
meta_data["date_first_available"] = pd.to_datetime(meta_data["date_first_available"], errors='coerce', infer_datetime_format=True)

# Identify rows with conversion issues
print("Invalid dates in reviews:", reviews["review_date"].isna().sum())
print("Invalid dates in meta_data:", meta_data["date_first_available"].isna().sum())

# Optionally drop rows with NaT values after conversion
reviews.dropna(subset=["review_date"], inplace=True)
meta_data.dropna(subset=["date_first_available"], inplace=True)

# Merge datasets
merged_df = reviews.merge(meta_data, on="parent_asin", how="left")

print("Merged Data Preview:")
display(merged_df.head())


<ipython-input-28-6419238e334a>:2: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  reviews["review_date"] = pd.to_datetime(reviews["review_date"], errors='coerce', infer_datetime_format=True)
<ipython-input-28-6419238e334a>:5: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  meta_data["date_first_available"] = pd.to_datetime(meta_data["date_first_available"], errors='coerce', infer_datetime_format=True)


Invalid dates in reviews: 0
Invalid dates in meta_data: 2306
Merged Data Preview:


/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,rating,title_x,text,helpful_vote,parent_asin,verified_purchase,review_date,main_category,title_y,average_rating,rating_number,price,store,date_first_available
0,4.0,12 mg is 12 on the periodic table people! Mg f...,This review is more to clarify someone else’s ...,3,B07TDSJZMR,True,2020-02-06,Health & Personal Care,High Potency Magnesium Citrate Capsules 1000mg...,4.5,470.0,NaN,Life Nutrition,2019-08-11
1,5.0,Save the lanet using less plastic.,Love these easy multitasking bleach tablets. B...,3,B08637FWWF,True,2020-11-02,Health & Personal Care,Evolve Concentrated Bleach Tablets - 32-ct Bot...,4.8,181.0,NaN,Evolve,2019-09-12
2,5.0,Fantastic,I have been suffering a couple months with hee...,0,B07KJVGNN5,True,2019-07-24,Health & Personal Care,"Dr. Foot's Gel Heel Protectors, Plantar Fascii...",3.6,78.0,NaN,Dr.Foot,2018-11-15
3,4.0,It holds the water and makes bubbles. That's ...,"It's cheap and it does what I wanted. The ""ma...",7,B092RP73CX,True,2022-09-04,Health & Personal Care,"Homedics Bubble Bliss Deluxe-Foot Spa, Heat Ma...",4.4,8312.0,NaN,Homedics,2019-09-17
4,1.0,Not for me,Didn't do a thing for me. Not saying they don'...,0,B08KYJLF5T,True,2022-01-20,Health & Personal Care,Brain Supplement 1053mg - Premium Nootropic Br...,4.1,94.0,NaN,Nature's Nutrition,2020-12-08


In [4]:
!pip install torch torchtext

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 21.6 MB/s eta 0:00:0000:0100:01


## **Loading the Libraries**

In [29]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report

In [30]:
# 🔹 1️⃣ Load & Preprocess Data
reviews["text"] = reviews["text"].fillna("")  # Fill missing values
reviews["label"] = reviews["rating"].apply(lambda x: 1 if x >= 3 else 0) 

In [31]:
X_train, X_test, y_train, y_test = train_test_split(reviews["text"], reviews["label"], test_size=0.2, random_state=42)


## **Tokenization**

In [32]:
# 🔹 2️⃣ Tokenization & Padding
max_words = 10000   # Vocabulary size
max_len = 200       # Max sequence length

In [33]:
tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)

In [34]:
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

X_train_pad = pad_sequences(X_train_seq, maxlen=max_len, padding="post", truncating="post")
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len, padding="post", truncating="post")

In [35]:
y_train, y_test = np.array(y_train), np.array(y_test)


In [36]:
# 🔹 3️⃣ Build TextCNN Model
def build_textcnn(vocab_size, embedding_dim=128, input_length=200):
    inputs = Input(shape=(input_length,))
    embedding = Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=input_length)(inputs)

    conv1 = Conv1D(filters=128, kernel_size=3, activation="relu")(embedding)
    pool1 = GlobalMaxPooling1D()(conv1)

    conv2 = Conv1D(filters=128, kernel_size=4, activation="relu")(embedding)
    pool2 = GlobalMaxPooling1D()(conv2)

    conv3 = Conv1D(filters=128, kernel_size=5, activation="relu")(embedding)
    pool3 = GlobalMaxPooling1D()(conv3)

    merged = tf.keras.layers.concatenate([pool1, pool2, pool3])
    dropout = Dropout(0.5)(merged)
    dense = Dense(32, activation="relu")(dropout)
    outputs = Dense(1, activation="sigmoid")(dense)

    model = Model(inputs=inputs, outputs=outputs)
    model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
    return model


In [37]:
model = build_textcnn(vocab_size=max_words)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


## **Training**

In [38]:
history = model.fit(X_train_pad, y_train, validation_data=(X_test_pad, y_test), epochs=5, batch_size=32)


Epoch 1/5
12353/12353 ━━━━━━━━━━━━━━━━━━━━ 63s 5ms/step - accuracy: 0.8926 - loss: 0.2520 - val_accuracy: 0.9204 - val_loss: 0.1930
Epoch 2/5
12353/12353 ━━━━━━━━━━━━━━━━━━━━ 61s 5ms/step - accuracy: 0.9249 - loss: 0.1803 - val_accuracy: 0.9234 - val_loss: 0.1822
Epoch 3/5
12353/12353 ━━━━━━━━━━━━━━━━━━━━ 61s 5ms/step - accuracy: 0.9350 - loss: 0.1591 - val_accuracy: 0.9236 - val_loss: 0.1824
Epoch 4/5
12353/12353 ━━━━━━━━━━━━━━━━━━━━ 61s 5ms/step - accuracy: 0.9418 - loss: 0.1433 - val_accuracy: 0.9228 - val_loss: 0.1893
Epoch 5/5
12353/12353 ━━━━━━━━━━━━━━━━━━━━ 61s 5ms/step - accuracy: 0.9481 - loss: 0.1297 - val_accuracy: 0.9218 - val_loss: 0.1906


## **Model Evaluation**

In [39]:
# 🔹 5️⃣ Evaluate Model
y_pred = (model.predict(X_test_pad) > 0.5).astype("int32")
print("\nClassification Report:\n", classification_report(y_test, y_pred, target_names=["Negative", "Positive"]))

3089/3089 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step

Classification Report:
               precision    recall  f1-score   support

    Negative       0.80      0.82      0.81     19904
    Positive       0.95      0.95      0.95     78921

    accuracy                           0.92     98825
   macro avg       0.88      0.88      0.88     98825
weighted avg       0.92      0.92      0.92     98825



In [24]:
model.save("textcnn_sentiment.h5")
tokenizer_json = tokenizer.to_json()
with open("tokenizer.json", "w") as f:
    f.write(tokenizer_json)

print("Model and tokenizer saved successfully!")

Model and tokenizer saved successfully!


In [40]:
# 🔹 Calculate Accuracy
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")


Accuracy: 0.9218


## **CNN+BiLSTM**

In [41]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Bidirectional, LSTM, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [42]:
# 🔹 Load and preprocess dataset
reviews["text"] = reviews["text"].fillna("")
reviews["label"] = reviews["rating"].apply(lambda x: 1 if x >= 3 else 0)

In [43]:
X_train, X_test, y_train, y_test = train_test_split(reviews["text"], reviews["label"], test_size=0.2, random_state=42)

In [44]:
# 🔹 Tokenization
max_words = 20000  # Vocabulary size
max_len = 200  # Max sequence length

In [45]:
tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)

In [46]:
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

In [47]:
# 🔹 Padding sequences
X_train_pad = pad_sequences(X_train_seq, maxlen=max_len, padding="post", truncating="post")
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len, padding="post", truncating="post")

In [48]:
y_train, y_test = np.array(y_train), np.array(y_test)


In [52]:
# 🔹 Build Hybrid CNN + BiLSTM Model
model = Sequential([
    Embedding(input_dim=max_words, output_dim=128, input_length=max_len),
    Conv1D(64, kernel_size=5, activation="relu", padding="same"),
    Bidirectional(LSTM(64, return_sequences=True)),  # Return sequences to maintain 3D input
    GlobalMaxPooling1D(),  # Now it can safely reduce dimensions
    Dropout(0.5),
    Dense(32, activation="relu"),
    Dense(1, activation="sigmoid")
])

In [53]:
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])


In [54]:
history = model.fit(X_train_pad, y_train, validation_data=(X_test_pad, y_test), epochs=5, batch_size=32)


Epoch 1/5
12353/12353 ━━━━━━━━━━━━━━━━━━━━ 222s 17ms/step - accuracy: 0.8980 - loss: 0.2412 - val_accuracy: 0.9253 - val_loss: 0.1833
Epoch 2/5
12353/12353 ━━━━━━━━━━━━━━━━━━━━ 214s 17ms/step - accuracy: 0.9331 - loss: 0.1632 - val_accuracy: 0.9259 - val_loss: 0.1761
Epoch 3/5
12353/12353 ━━━━━━━━━━━━━━━━━━━━ 214s 17ms/step - accuracy: 0.9438 - loss: 0.1394 - val_accuracy: 0.9260 - val_loss: 0.1782
Epoch 4/5
12353/12353 ━━━━━━━━━━━━━━━━━━━━ 214s 17ms/step - accuracy: 0.9532 - loss: 0.1194 - val_accuracy: 0.9234 - val_loss: 0.1925
Epoch 5/5
12353/12353 ━━━━━━━━━━━━━━━━━━━━ 214s 17ms/step - accuracy: 0.9612 - loss: 0.1009 - val_accuracy: 0.9225 - val_loss: 0.2069


In [55]:
# 🔹 Evaluate Model
y_pred = (model.predict(X_test_pad) > 0.5).astype("int32")
accuracy = accuracy_score(y_test, y_pred)
print(f"Test Accuracy: {accuracy:.4f}")

3089/3089 ━━━━━━━━━━━━━━━━━━━━ 16s 5ms/step
Test Accuracy: 0.9225


In [56]:
# 🔹 Save Model
model.save("hybrid_cnn_bilstm_sentiment.h5")
tokenizer_json = tokenizer.to_json()
with open("tokenizer.json", "w") as f:
    f.write(tokenizer_json)

print("Model and tokenizer saved successfully!")

Model and tokenizer saved successfully!


In [57]:
from sklearn.metrics import classification_report

print("\nClassification Report:\n", classification_report(y_test, y_pred, target_names=["Negative", "Positive"]))



Classification Report:
               precision    recall  f1-score   support

    Negative       0.83      0.78      0.80     19904
    Positive       0.94      0.96      0.95     78921

    accuracy                           0.92     98825
   macro avg       0.89      0.87      0.88     98825
weighted avg       0.92      0.92      0.92     98825

